# Connect companies universe with PE_backed

In [73]:
import pandas as pd
import numpy as np

df_companies = pd.read_csv(r"C:\Users\minha\OneDrive\Courses\Thesis\final _datasets\companies_universe_repiq.csv")
df_PE_backed = pd.read_csv(r"C:\Users\minha\OneDrive\Courses\Thesis\final _datasets\PE_backed.csv")

# Replace NaN with empty lists in the 'PE_Firm' column
df_PE_backed['PE_Firm'] = df_PE_backed['PE_Firm'].apply(lambda x: [] if pd.isna(x) else x)

In [74]:
PE_id_list = list(df_PE_backed["Target ID"].unique())
companies_uni_id = list(df_companies["SP_ENTITY_ID"].unique())

# First, convert PE_id_list to a set for efficiency
PE_id_set = set(PE_id_list)

# Filter df_companies to keep only rows where SP_ENTITY_ID is NOT in PE_id_list
df_companies_cleaned = df_companies[~df_companies["SP_ENTITY_ID"].isin(PE_id_set)].copy()
df_companies_cleaned.reset_index(drop=True, inplace=True)


In [75]:
# Years you want
years = list(range(2007, 2021))

# Create a list to hold new rows
new_rows = []

# Iterate over cleaned companies
for idx, row in df_companies_cleaned.iterrows():
    for year in years:
        new_rows.append({
            "Company": row["SP_ENTITY_NAME"],  
            "Target ID": row["SP_ENTITY_ID"],
            "Target": row["SP_ENTITY_NAME"],
            "Year": year,
            "PE_Firm": [],
            "PE-backed": 0,
            "reprisk_id": row["reprisk_id"]  
        })

# Create a DataFrame from the new rows
df_companies_expanded = pd.DataFrame(new_rows)

# Combine with PE-backed dataset
df_id_repiq = pd.concat([df_PE_backed, df_companies_expanded], ignore_index=True)


# Calculation for yearly Reprisk

In [76]:
df_reprisk = pd.read_csv(r"C:\Users\minha\OneDrive\Courses\Thesis\RepRisk Index.csv")

columns_to_drop = [
    'name', 
    'headquarter_country', 
    'url', 
    'all_ISINs', 
    'primary_ISIN', 
    'No_reported_risk_exposure'
]

# Drop the columns
df_reprisk = df_reprisk.drop(columns=columns_to_drop)

# Ensure your 'date' column is in datetime format
df_reprisk['date'] = pd.to_datetime(df_reprisk['date'])
df_reprisk['peak_RRI_date'] = pd.to_datetime(df_reprisk['peak_RRI_date'], errors='coerce')

# Create a 'year' column
df_reprisk['year'] = df_reprisk['date'].dt.year

## yearly calculation

In [77]:
def mode(series):
    if series.dropna().empty:
        return None
    return series.mode().iloc[0] if not series.mode().empty else None

# Now group by RepRisk_ID and year
df_reprisk_yearly = df_reprisk.groupby(['RepRisk_ID', 'year']).agg({
    'current_RRI': 'mean',        # Take the maximum risk index in the year
    'RRI_trend': 'sum',          # Average trend over the year
    'peak_RRI': 'max',            # Max peak_RRI
    'peak_RRI_date': 'max',     #  Latest date with peak RRI
    'RepRisk_rating': mode,     
    'country_sector_average': 'mean',  # Average across year
    'environmental_percentage': 'last', # Take last known percentage
    'social_percentage': 'last',
    'governance_percentage': 'last',
    'headquarter_country_code': 'first',  # Same across the year
    'sectors': 'first'                   # Assuming sector doesn't change
}).reset_index()


## yearly snapshot

In [78]:
# Sort by RepRisk_ID, year, date (latest date on top)
df_reprisk = df_reprisk.sort_values(['RepRisk_ID', 'year', 'date'], ascending=[True, True, False])

# Pick the first (latest) row in each (RepRisk_ID, year) group
df_reprisk_yearly_snapshot = (
    df_reprisk
    .groupby(['RepRisk_ID', 'year'], as_index=False)
    .first()
)

# Drop 'date' column if you don't want it
df_reprisk_yearly_snapshot = df_reprisk_yearly_snapshot.drop(columns=['date'])


In [79]:
# Perform the merge
merged_index_snapshot_df = pd.merge(
    df_id_repiq,
    df_reprisk_yearly_snapshot,
    left_on=['Year', 'reprisk_id'],
    right_on=['year', 'RepRisk_ID'],
    how='inner'
)

# Optional: if you want to drop the duplicate 'year' and 'RepRisk_ID' columns after the merge
merged_index_snapshot_df = merged_index_snapshot_df.drop(columns=['year', 'RepRisk_ID'])


# Connect incidents

In [80]:
df_incidents_w_id= pd.read_csv(r"C:\Users\minha\OneDrive\Courses\Thesis\RepRisk Incidents.csv")
df_incidents = pd.read_csv(r"demo/not_filtered_incidents.csv")

In [81]:
import re

# --- 1. Clean names (remove punctuation, normalize spaces) ---
suffixes_pattern = r'\b(llc|inc|ltd|corp|gmbh|spa|sas|plc|co|company|limited)\b'

def clean_name(name):
    if isinstance(name, str):
        name = name.lower()
        name = re.sub(r'[^\w\s]', '', name)         # Remove punctuation
        name = re.sub(suffixes_pattern, '', name)   # Remove suffixes
        name = re.sub(r'\s+', ' ', name)             # Normalize spaces
        return name.strip()
    return ""

In [82]:
df_incidents['cleaned_company_name'] = df_incidents['company_name'].apply(clean_name)

In [83]:
# Create a mapping from cleaned company name to the first reprisk_id found
name_to_id_map = df_incidents_w_id.drop_duplicates('company_name').set_index('company_name')['reprisk_id']

# Map the reprisk_id to df_incidents using the cleaned_company_name column
df_incidents['reprisk_id'] = df_incidents['company_name'].map(name_to_id_map)


In [86]:
# First, ensure the year columns are properly named/aligned for merging
df_incidents = df_incidents.rename(columns={'year': 'Year'})  # rename 'year' in df_incidents to 'Year' if necessary

# Now perform the merge
df_reprisk_yearly_snapshot_w_incidents = pd.merge(
    merged_index_snapshot_df,
    df_incidents,
    how='left', 
    on=['reprisk_id', 'Year'])

df_reprisk_yearly_snapshot_w_incidents = df_reprisk_yearly_snapshot_w_incidents.drop(columns=['cleaned_company_name', 'company_name'])

In [102]:
df_reprisk_yearly_snapshot_w_incidents.to_csv("final_dataset_NEWEST.csv")

# Examine datasets

In [95]:
PE_unique = df_id_repiq[df_id_repiq["PE-backed"] == 1]["reprisk_id"].nunique()
no_PE_unique = df_id_repiq[df_id_repiq["PE-backed"] == 0]["reprisk_id"].nunique()
print(PE_unique)
print(no_PE_unique)

1059
4613


In [87]:
df_reprisk_yearly_snapshot_w_incidents.columns

Index(['Company', 'Target ID', 'Target', 'Year', 'PE_Firm', 'PE-backed',
       'reprisk_id', 'current_RRI', 'RRI_trend', 'peak_RRI', 'peak_RRI_date',
       'RepRisk_rating', 'country_sector_average', 'environmental_percentage',
       'social_percentage', 'governance_percentage',
       'headquarter_country_code', 'sectors', 'Total Climate Incidents',
       'Max Severity Score', 'GHG-Related Incidents',
       'Pollution-Related Incidents', 'UNGC Principle 7 Violations',
       'UNGC Principle 8 Violations', 'UNGC Principle 9 Violations'],
      dtype='object')

In [96]:
unique_id = df_reprisk_yearly_snapshot_w_incidents["reprisk_id"].nunique()
unique_id

1061

In [91]:
import pandas as pd

# Filter the DataFrame to keep only non-NA and non-zero values
filtered_df = df_reprisk_yearly_snapshot_w_incidents[
    df_reprisk_yearly_snapshot_w_incidents["Total Climate Incidents"].notna()
    & (df_reprisk_yearly_snapshot_w_incidents["Total Climate Incidents"] != 0)
]

# Group by 'RepRisk_ID' and count valid entries
result = (
    filtered_df.groupby("reprisk_id")["Total Climate Incidents"].count().reset_index()
)

# Rename the count column for clarity (optional)
result = result.rename(
    columns={"Total Climate Incidents": "Non-NA and Non-Zero Climate Incidents Count"}
)

print(result)

     reprisk_id  Non-NA and Non-Zero Climate Incidents Count
0            30                                           14
1            76                                           13
2            80                                           14
3            85                                           10
4            91                                           13
..          ...                                          ...
372     1507307                                            1
373     1692694                                            1
374     1709987                                            1
375     1772214                                            1
376     2022922                                            1

[377 rows x 2 columns]


In [101]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

# Drop rows with missing values in key columns
df = df_reprisk_yearly_snapshot_w_incidents.dropna(subset=['PE-backed', 'Total Climate Incidents'])

# Regression with year dummies using Q() to quote variable names with spaces
model = smf.ols('Q("Total Climate Incidents") ~ Q("PE-backed") + C(Year)', data=df).fit()

# Print regression summary
print(model.summary())

                                 OLS Regression Results                                 
Dep. Variable:     Q("Total Climate Incidents")   R-squared:                       0.006
Model:                                      OLS   Adj. R-squared:                 -0.005
Method:                           Least Squares   F-statistic:                    0.5481
Date:                          Fri, 02 May 2025   Prob (F-statistic):              0.905
Time:                                  12:03:37   Log-Likelihood:                -3831.7
No. Observations:                          1229   AIC:                             7693.
Df Residuals:                              1214   BIC:                             7770.
Df Model:                                    14                                         
Covariance Type:                      nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------------